In [12]:
import numpy as np
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# VMFI Data processing pipeline

This workbook aims to emulate the current data processing pipeline that occurs in VMFI pipeline. The logic and processing is largely based on the following document [Insights data portal - Data sources and sql analysis](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) and will stay true to this document even if the existing stored procedures are doing something different. This will form the basis of a gap analysis going forward. 

All data loaded in the following workbook comes from the set of CSV files in the `data` folder alongside this workbook. These datasets are for the most part from the list at the start of the linked document. However, because there is additional standing data required to fully implement the pipeline then this data has been exported from the development VMFI pipeline database. These files are currently: 

| File name | DB Table |
|:----------|----------|
|standing_data_cdc.csv | standing_data.cdc |

In [13]:
import pandas as pd
import mappings as mappings
import schemas as schemas
import datetime

from pathlib import Path
Path("output/pre-processing").mkdir(parents=True, exist_ok=True)

current_year = 2022
accounts_return_period_start_date = datetime.date(current_year - 1, 9, 10)
academy_year_start_date = datetime.date(current_year - 1, 9, 1)
academy_year_end_date = datetime.date(current_year, 8, 30)
maintained_schools_year_start_date = datetime.date(current_year, 4, 1)
maintained_schools_year_end_date = datetime.date(current_year, 3, 31)

## CDC data load and preparation

School buildings condition dataset. Based on the surveys performed throughout 2018-2019.

The data in the file `data/standing_data_cdc.csv` is just an export of the data in `standing_data.cdc` table. Without the Year and Import ID fields. In future this will likely have to be read directly from the source database as per [this document.](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) 

In [14]:
cdc = pd.read_csv('data/standing_data_cdc.csv', encoding='utf8', index_col=schemas.cdc_index_col, usecols=schemas.cdc.keys(), dtype=schemas.cdc)

cdc['Total Internal Floor Area'] = cdc.groupby(by=['URN'])['GIFA'].sum()
cdc['Proportion Area'] = (cdc['GIFA'] / cdc['Total Internal Floor Area'])
cdc['Indicative Age'] = cdc['Block Age'].fillna('').map(mappings.map_block_age).astype('Int64')
cdc['Age Score'] = cdc['Proportion Area'] * (current_year - cdc['Indicative Age'])
cdc['Score'] = cdc.groupby(by=['URN'])['Age Score'].sum()
cdc = cdc[['Total Internal Floor Area', 'Score']].drop_duplicates()

In [15]:
cdc.to_csv('output/pre-processing/cdc.csv')
cdc

,Total Internal Floor Area,Score
URN,,
100150,2803.0,48.358188
100162,2105.0,133.162945
100164,2934.0,97.0
100166,2040.0,91.705882
105304,1602.0,35.752809
...,...,...
144913,3111.0,16.704275
144917,2620.0,78.412214
105623,3382.0,7.0


## School Census data load

*Pupil Census* - DfE data collection providing information about school and pupil characteristics, for example percentage of pupils claiming free school`z meals, or having English as their second language. 

*Workforce census* - Single reference for all school workforce statistics based on staff working in publicly funded schools in England.

The following code loads both the workforce and pupil census data and preforms an `inner` join by URN on the data sets.

In [16]:
school_workforce_census = pd.read_excel('data/School_Tables_School_Workforce_Census_2022.xlsx', header=5, index_col=schemas.workforce_census_index_col, usecols=schemas.workforce_census.keys(), dtype=schemas.workforce_census, na_values=["x","u"], keep_default_na=True).drop_duplicates()

school_pupil_census = pd.read_csv('data/standing_data_census_pupils.csv', encoding='utf8', index_col=schemas.pupil_census_index_col, usecols=schemas.pupil_census.keys(), dtype=schemas.pupil_census).drop_duplicates()

census = school_pupil_census.join(school_workforce_census, on='URN', how='inner', rsuffix='_pupil', lsuffix='_workforce')
         

In [17]:
census.to_csv('output/pre-processing/census.csv')
census

,full time pupils,headcount of pupils,% of pupils known to be eligible for and claiming free school me,number of pupils whose first language is known or believed to be other than English,Total School Workforce (Full-Time Equivalent),Total Number of Teachers (Full-Time Equivalent),Pupil: Teacher Ratio (Full-Time Equivalent of qualified and unqualified teachers),Number of Vacant Teacher Posts
URN,,,,,,,,
141334,325.0,325.0,33.8,93.0,34.17,13.11,24.8,0.0
141396,599.0,642.0,23.4,236.0,82.47,34.00,18.3,0.0
141397,465.0,500.0,33.2,127.0,72.81,24.55,19.7,0.0
142223,1050.0,1117.0,5.1,343.0,99.66,47.12,23.0,0.0
144396,196.0,217.0,56.7,29.0,25.57,11.39,18.1,0.0
...,...,...,...,...,...,...,...,...
104642,421.0,421.0,2.4,14.0,34.47,15.80,26.6,0.0
104643,426.0,432.0,3.5,13.0,39.89,17.40,24.7,0.0
104645,237.0,237.0,32.9,43.0,26.47,12.40,19.1,0.0


## Special Education Needs (SEN) data load and preparation

Special educational needs dataset. Contains information about the number of pupils, who require various SEN provisions. This loads the `SEN` data, which originates from [here](https://explore-education-statistics.service.gov.uk/find-statistics/special-educational-needs-in-england#dataDownloads-1)

In [18]:
sen = pd.read_csv('data/SEN.csv', encoding='cp1252', index_col=schemas.sen_index_col, dtype=schemas.sen, usecols=schemas.sen.keys())
sen['% of pupils with special educational needs support'] = (sen['EHC plan'] / sen['Total pupils']) * 100.0

In [19]:
sen.to_csv("output/pre-processing/sen.csv")
sen

,Total pupils,SEN support,EHC plan,EHC_Primary_need_spld,EHC_Primary_need_mld,EHC_Primary_need_sld,EHC_Primary_need_pmld,EHC_Primary_need_semh,EHC_Primary_need_slcn,EHC_Primary_need_hi,...,SUP_Primary_need_semh,SUP_Primary_need_slcn,SUP_Primary_need_hi,SUP_Primary_need_vi,SUP_Primary_need_msi,SUP_Primary_need_pd,SUP_Primary_need_asd,SUP_Primary_need_oth,SUP_Primary_need_nsa,% of pupils with special educational needs support
URN,,,,,,,,,,,,,,,,,,,,,
100000,271,59,8,0,0,0,0,1,1,0,...,8,30,2,0,2,0,4,0,7,2.95203
100001,739,22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
100002,269,22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
100003,1045,145,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
100005,136,23,2,0,0,0,1,0,0,0,...,0,0,0,0,0,1,21,1,0,1.470588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149557,41,2,3,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,7.317073
149632,1291,136,58,2,1,0,0,3,15,4,...,17,10,4,2,1,3,6,16,34,4.492641
149633,86,7,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,1,0,0.0


## AR Data load and preparation

This loads the Annual accounts return dataset and the corresponding mapping file. This extract only contains benchmarking section, which consists of submissions of costs, income, and balances of individual academies.

The mapping file, contains the mapping from AR4 cell references to cost categories and descriptions.

In [20]:
ar_cell_mapping = pd.read_csv('data/AR_cell_mapping.csv', encoding='utf8', index_col=schemas.ar_cell_mapping_index_col, usecols=schemas.ar_cell_mapping.keys(), dtype=schemas.ar_cell_mapping)

ar_raw = pd.read_csv('data/AR_raw.csv', encoding='utf8', index_col=schemas.ar_index_col, usecols=schemas.ar.keys(), dtype=schemas.ar)

ar = ar_raw.reset_index().merge(ar_cell_mapping, right_on='cell', left_on='aruniquereference').set_index(schemas.ar_index_col)

pfi_schools = ar.groupby('academyupin')[ar['aruniquereference' == 'BAE310-T']]

In [32]:
ar.to_csv('output/pre-processing/ar.csv')
ar

,trustupin,companynumber,aruniquereference,value,Description L1,Metric,Metric ID,Cost Pool,Presentation name,Cost Pool ID
academyupin,,,,,,,,,,
141615,137302,8212263,BAE010-T,603.0,Staff costs,Teaching staff,1,Teaching and Teaching support staff,Teaching and Teaching support staff,1
141615,137302,8212263,BAE020-T,0.0,Staff costs,Supply teaching staff,5,Teaching and Teaching support staff,Teaching and Teaching support staff,1
141615,137302,8212263,BAE030-T,273.0,Staff costs,Education support staff,2,Teaching and Teaching support staff,Teaching and Teaching support staff,1
141615,137302,8212263,BAE040-T,49.0,Staff costs,Administrative and clerical staff,6,Non-educational support staff,Non-educational support staff,2
141615,137302,8212263,BAE050-T,41.0,Staff costs,Premises staff,14,Premises,Premises,5
...,...,...,...,...,...,...,...,...,...,...
<NA>,136943,7973953,BTI110,0.0,Self-generated income,Income from catering,33,Catering,Catering income,10
<NA>,136943,7973953,BTB030,0.0,Closing balance,Closing balance (Restricted and Unrestricted F...,0,<NA>,<NA>,0
<NA>,137730,8476149,BTITOT,2455.0,Total revenue income,Total revenue income,0,<NA>,<NA>,0


In [46]:
pfi_schools = ar[ar['aruniquereference'] == 'BAE310-T'][['value']].map(lambda x: x > 0)
pfi_schools

,value
academyupin,
141615,False
151709,False
121200,False
121205,False
121209,False
...,...
119734,False
119784,False
163823,False


## Academy and maintained schools data load and preparation

This reads the main GIAS data (edubasealldataYYYYMMDD file) and the associated links file (links_edubasealldataYYYYMMDD file). This is taken from the [GIAS Service](https://get-information-schools.service.gov.uk/help)

Other columns are tidied up by asserting the correct type for that column. This is tidying phase is largly because on load integer columns will be inferred to be a float as opposed to an integer.

In [22]:
gias = pd.read_csv('data/edubasealldata20240312.csv', encoding='cp1252', 
                   index_col=schemas.gias_index_col, usecols=schemas.gias.keys(), dtype=schemas.gias)

gias_links = pd.read_csv('data/links_edubasealldata20240312.csv', encoding='cp1252', 
                         index_col=schemas.gias_links_index_col, usecols=schemas.gias_links.keys(), dtype=schemas.gias_links)

# GIAS transformations
gias['LA Establishment Number'] = gias['LA (code)'] + '-' + gias['EstablishmentNumber'].astype('string')
gias['LA Establishment Number'] = gias['LA Establishment Number'].astype('string')

gias['OpenDate'] = pd.to_datetime(gias['OpenDate'], dayfirst=True, format='mixed')
gias['CloseDate'] = pd.to_datetime(gias['CloseDate'], dayfirst=True, format='mixed')
gias['SchoolWebsite'] = gias['SchoolWebsite'].fillna('').map(mappings.map_school_website)
gias['Boarders (name)'] = gias['Boarders (name)'].fillna('').map(mappings.map_boarders)
gias['OfstedRating (name)'] = gias['OfstedRating (name)'].fillna('').map(mappings.map_ofsted_rating)
gias['NurseryProvision (name)'] = gias['NurseryProvision (name)'].fillna('')
gias['OfficialSixthForm (name)'] = gias['OfficialSixthForm (name)'].fillna('').map(mappings.map_sixth_form)
gias['AdmissionsPolicy (name)'] = gias['AdmissionsPolicy (name)'].fillna('').map(mappings.map_admission_policy)
gias['Head Name'] = gias['HeadTitle (name)'] + ' ' + gias['HeadFirstName'] + ' ' + gias['HeadLastName']

In the following cell, we find all the predecessor and merged links. The links are then Ranked by URN and order by 'Link Established Date'. The linked GAIS data in then joined to the base GIAS data. This creates the overall school data set. This dataset is then filtered for schools that are open (CloseDate is null) and the schools with nested links that are Ranked 1.

In [23]:
gias_links = gias_links[
    gias_links['LinkType'].isin(['Predecessor', 
                                 'Predecessor - amalgamated', 
                                 'Predecessor - Split School', 
                                 'Predecessor - merged', 
                                 'Merged - expansion of school capacity', 
                                 'Merged - change in age range'])
].sort_values(by='LinkEstablishedDate', ascending=False)

gias_links['Rank'] = gias_links.groupby('URN').cumcount() + 1
gias_links['Rank'] = gias_links['Rank'].astype('Int64')

schools = gias.join(gias_links, on='URN', how='left', rsuffix='_links', lsuffix='_school').sort_values(by='URN')

schools = schools[
    schools['CloseDate'].isna() & ((schools['Rank'] == 1) | (schools['Rank'].isna()))
].drop(columns=['LinkURN', 'LinkName', 'LinkType', 'LinkEstablishedDate', 'Rank'])

In [24]:
schools.to_csv('output/pre-processing/schools.csv')
schools.sort_index()

,LA (code),LA (name),EstablishmentNumber,EstablishmentName,TypeOfEstablishment (code),TypeOfEstablishment (name),EstablishmentStatus (code),EstablishmentStatus (name),OpenDate,CloseDate,...,UrbanRural (name),BoardingEstablishment (name),PreviousLA (code),PreviousLA (name),PreviousEstablishmentNumber,OfstedRating (name),MSOA (code),LSOA (code),LA Establishment Number,Head Name
URN,,,,,,,,,,,,,,,,,,,,,
100000,201,City of London,3614,The Aldgate School,2,Voluntary aided school,1,Open,NaT,NaT,...,(England/Wales) Urban major conurbation,<NA>,999,<NA>,<NA>,Outstanding,E02000001,E01032739,201-3614,Miss Alexandra Allan
100001,201,City of London,6005,City of London School for Girls,11,Other independent school,1,Open,1920-01-01,NaT,...,(England/Wales) Urban major conurbation,Does not have boarders,999,<NA>,<NA>,,E02000001,E01000002,201-6005,Mrs Jenny Brown
100002,201,City of London,6006,St Paul's Cathedral School,11,Other independent school,1,Open,1939-01-01,NaT,...,(England/Wales) Urban major conurbation,Has boarders,999,<NA>,<NA>,,E02000001,E01032739,201-6006,<NA>
100003,201,City of London,6007,City of London School,11,Other independent school,1,Open,1919-01-01,NaT,...,(England/Wales) Urban major conurbation,Does not have boarders,999,<NA>,<NA>,,E02000001,E01032739,201-6007,Mr Alan Bird
100005,202,Camden,1048,Thomas Coram Centre,15,Local authority nursery school,1,Open,NaT,NaT,...,(England/Wales) Urban major conurbation,<NA>,999,<NA>,<NA>,Outstanding,E02007115,E01000937,202-1048,Ms Perina Holness
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402468,679,Monmouthshire,5500,King Henry viii 3-19 School,30,Welsh establishment,1,Open,2023-09-01,NaT,...,<NA>,<NA>,999,<NA>,<NA>,,999999999,999999999,679-5500,<NA>
402469,681,Cardiff,2333,Ysgol Gynradd Groes-Wen Primary,30,Welsh establishment,1,Open,2023-09-01,NaT,...,(England/Wales) Rural village,<NA>,999,<NA>,<NA>,,W02000380,W01001729,681-2333,<NA>
402470,668,Pembrokeshire,2398,Ysgol Bro Penfro,30,Welsh establishment,4,Proposed to open,2024-09-01,NaT,...,(England/Wales) Rural town and fringe,<NA>,999,<NA>,<NA>,,W02000140,W01000607,668-2398,<NA>


Merge required GIAS, census, sen, cdc and arr data with the base academy data

In [27]:
academies_list = pd.read_csv('data/master_list_raw.csv', encoding='utf8', index_col=schemas.academy_master_list_index_col, dtype=schemas.academy_master_list, usecols=schemas.academy_master_list.keys()).rename(columns={'UKPRN': 'Academy UKPRN'})

academies_base = academies_list.merge(schools.reset_index(), left_index=True, right_on='LA Establishment Number').set_index('URN')

academies = (academies_base.merge(census, on='URN', how='left')
             .merge(sen, on='URN', how='left')
             .merge(cdc, on='URN', how='left'))

academies['Type of Provision - Phase'] = academies.apply(lambda df: mappings.map_academy_phase_type(df['TypeOfEstablishment (code)'], df['Type of Provision - Phase']), axis=1)

# Bizarre I shouldn't need this as this is coming from the original GIAS dataset, but I seem to have to do this twice. 
academies['NurseryProvision (name)'] = academies['NurseryProvision (name)'].fillna('')
academies['NurseryProvision (name)'] = academies.apply(lambda df: mappings.map_nursery(df['NurseryProvision (name)'], df['Type of Provision - Phase']), axis=1)

academies['Status'] = academies.apply(lambda df: mappings.map_academy_status(pd.to_datetime(df['Date left or closed if in period']), 
                                                                             pd.to_datetime(df['Valid to']), 
                                                                             pd.to_datetime(df['OpenDate']), 
                                                                             pd.to_datetime(df['CloseDate']), 
                                                                             pd.to_datetime(accounts_return_period_start_date), pd.to_datetime(academy_year_start_date), pd.to_datetime(academy_year_end_date)), axis=1)

academies.rename(columns={'UKPRN_x':'UKPRN'}, inplace=True)

In [28]:
academies.to_csv('output/pre-processing/academies.csv')
academies.sort_index()

,Company Registration Number,Incorporation Date,Academy Trust UPIN,Academy UKPRN,Academy Trust Name,Academy Name,Academy UPIN,Trust Type,Date Opened,LA Name,...,SUP_Primary_need_vi,SUP_Primary_need_msi,SUP_Primary_need_pd,SUP_Primary_need_asd,SUP_Primary_need_oth,SUP_Primary_need_nsa,% of pupils with special educational needs support,Total Internal Floor Area,Score,Status
URN,,,,,,,,,,,,,,,,,,,,,
101849,02369239,1989-04-06 00:00:00.0000000,134879,10058152,The BRIT School Limited,BRIT School for Performing Arts and Technology,118384,Single Academy Trust (SAT),1991-10-22 00:00:00.0000000,Croydon London Borough Council,...,1,0,16,32,0,0,3.435934,27058.0,43.834208,Open
105135,05210075,2004-08-19 00:00:00.0000000,135025,10058185,St Paul's Academy,St Paul's Academy - Greenwich,119110,Single Academy Trust (SAT),2005-09-01 00:00:00.0000000,Greenwich London Borough Council,...,1,0,1,14,3,7,2.5,11352.0,16.7463,Open
108420,04464331,2002-06-19 00:00:00.0000000,139672,10064192,Emmanuel Schools Foundation,Emmanuel College - Gateshead,118390,Multi Academy Trust (MAT),1991-04-05 00:00:00.0000000,Gateshead Council,...,4,0,3,28,11,23,0.692521,15903.0,37.0,Open
123627,02414699,1989-08-18 00:00:00.0000000,134882,10058155,Telford City Technology College Trust Limited,Thomas Telford School,118401,Single Academy Trust (SAT),1992-02-07 00:00:00.0000000,Telford and Wrekin Council,...,5,1,7,6,0,0,0.12945,28298.0,23.204891,Open
129342,07525820,2011-02-10 00:00:00.0000000,135906,10058513,Tove Learning Trust,Grace Academy Solihull,118615,Multi Academy Trust (MAT),2006-09-01 00:00:00.0000000,Solihull Metropolitan Borough Council,...,5,1,2,68,21,0,2.447164,17294.0,16.930612,Open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149222,07668923,2011-06-14 00:00:00.0000000,135225,10058887,Coast Academies,Cockington Primary School,163969,Multi Academy Trust (MAT),2013-09-01 00:00:00.0000000,Torbay Council,...,0,0,1,0,0,14,3.455285,NaN,<NA>,Closed in period
149222,10192252,2016-05-20 00:00:00.0000000,139657,10064173,Connect Academy Trust,Cockington Primary School,163969,Multi Academy Trust (MAT),2013-09-01 00:00:00.0000000,Torbay Council,...,0,0,1,0,0,14,3.455285,NaN,<NA>,(Re)opened in period
149299,09066969,2014-06-02 00:00:00.0000000,135834,10060816,Blessed Christopher Wharton Catholic Academy T...,"The Holy Family Catholic School, a Voluntary A...",162962,Multi Academy Trust (MAT),2022-08-01 00:00:00.0000000,Bradford Metropolitan District Council,...,2,0,6,10,2,0,3.944316,9429.0,41.71524,(Re)opened in period


Merge required census and cdc data to the maintained schools data set

In [29]:
maintained_schools_list = pd.read_csv('data/maintained_schools_raw.csv', encoding='utf8', index_col=schemas.maintained_schools_master_list_index_col, usecols=schemas.maintained_schools_master_list.keys(), dtype=schemas.maintained_schools_master_list)

maintained_schools_base = maintained_schools_list.merge(schools.reset_index(), left_index=True, right_on='URN').set_index('URN')

maintained_schools = maintained_schools_list.merge(schools.reset_index(), left_index=True, right_on='URN')

maintained_schools = (maintained_schools
                      .merge(census, on='URN', how='left')
                      .merge(cdc, on='URN', how='left'))

maintained_schools['Status'] = maintained_schools.apply(lambda df: mappings.map_maintained_school_status(df['OpenDate'], df['CloseDate'], df['Period covered by return (months)'], pd.to_datetime(maintained_schools_year_start_date), pd.to_datetime(maintained_schools_year_end_date)), axis=1)

maintained_schools.set_index('URN', inplace=True)

In [30]:
maintained_schools.to_csv('output/pre-processing/maintained_schools.csv')
maintained_schools

,Phase,Overall Phase,Lowest age of pupils,Highest age of pupils,Type,No Pupils,% of pupils eligible for FSM,Period covered by return (months),Did Not Supply flag,Federation,...,headcount of pupils,% of pupils known to be eligible for and claiming free school me,number of pupils whose first language is known or believed to be other than English,Total School Workforce (Full-Time Equivalent),Total Number of Teachers (Full-Time Equivalent),Pupil: Teacher Ratio (Full-Time Equivalent of qualified and unqualified teachers),Number of Vacant Teacher Posts,Total Internal Floor Area,Score,Status
URN,,,,,,,,,,,,,,,,,,,,,
100000,Infant and junior,Primary,3.0,11.0,Voluntary aided school,271.0,18.1,12,0,No,...,271.0,9.2,145.0,41.62,15.64,17.3,0.0,3628.0,142.0,Open
100005,Nursery,Nursery,2.0,5.0,Local authority nursery school,107.5,38.2,12,0,No,...,136.0,38.2,38.0,37.48,4.31,24.9,0.0,NaN,<NA>,Open
100006,Pupil referral unit,Pupil referral unit,11.0,16.0,Pupil referral unit,49.0,68.4,12,0,No,...,19.0,0.0,5.0,26.30,9.80,5.0,1.0,1523.0,59.797111,Open
100007,Pupil referral unit,Pupil referral unit,5.0,11.0,Pupil referral unit,19.0,100.0,12,0,No,...,19.0,0.0,4.0,31.58,15.80,1.2,0.0,NaN,<NA>,Open
100008,Infant and junior,Primary,3.0,11.0,Community school,350.0,55.1,12,0,No,...,350.0,53.1,306.0,45.22,18.60,18.8,0.0,2951.0,117.0,Open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131818,Infant and junior,Primary,3.0,11.0,Community school,417.0,52.0,12,0,No,...,429.0,36.4,35.0,54.27,25.00,16.7,0.0,2819.0,56.148634,Open
132176,Infant and junior,Primary,3.0,11.0,Voluntary aided school,278.0,45.9,12,0,No,...,292.0,13.0,140.0,32.97,14.40,19.3,2.0,2872.0,27.0,Open
132793,Infant and junior,Primary,4.0,11.0,Voluntary aided school,411.0,28.2,12,0,No,...,411.0,25.8,26.0,39.38,19.40,21.2,0.0,1639.0,117.0,Open
